In [1]:
from djstripe.models import *
from django.conf import settings
from backend.utility import *
from backend.stripe import stripe
from subscription.enum import *

In [2]:
u = User.objects.get(email='veyorokon@gmail.com')

In [3]:
account = SubscriptionAccount.objects.all()[1]

In [5]:
account.responsible_member

DoesNotExist: SubscriptionMember matching query does not exist.

In [7]:
u.djstripe_customer.subscription.id

'sub_Ek0NBc7clsXX73'

In [73]:
items = u.djstripe_subscription.items.all()
item = items[1]
dir(item)

['DoesNotExist',
 'Meta',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api_create',
 '_api_delete',
 '_attach_objects_hook',
 '_attach_objects_post_save_hook',
 '_check_column_name_clashes',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check

In [74]:
print(item.api_retrieve())

{
  "billing_thresholds": null,
  "created": 1553195763,
  "id": "si_Ek0NfWGSIJdNd0",
  "metadata": {},
  "object": "subscription_item",
  "plan": {
    "active": true,
    "aggregate_usage": null,
    "amount": 0,
    "billing_scheme": "per_unit",
    "created": 1553146706,
    "currency": "usd",
    "id": "plan_EjnCfmT2JdOt1a",
    "interval": "month",
    "interval_count": 1,
    "livemode": false,
    "metadata": {},
    "nickname": "Monthly",
    "object": "plan",
    "product": "prod_EjnBIJAleVzUNz",
    "tiers": null,
    "tiers_mode": null,
    "transform_usage": null,
    "trial_period_days": null,
    "usage_type": "licensed"
  },
  "quantity": 1,
  "subscription": "sub_Ek0NBc7clsXX73"
}


In [75]:
test = stripe.SubscriptionItem.retrieve('si_EkSI1DF2XNa055')
test

<SubscriptionItem subscription_item id=si_EkSI1DF2XNa055 at 0x7fe1b53fba40> JSON: {
  "billing_thresholds": null,
  "created": 1553299634,
  "id": "si_EkSI1DF2XNa055",
  "metadata": {},
  "object": "subscription_item",
  "plan": {
    "active": true,
    "aggregate_usage": null,
    "amount": 699,
    "billing_scheme": "per_unit",
    "created": 1553216533,
    "currency": "usd",
    "id": "plan_Ek5xhjdwNQYNEd",
    "interval": "month",
    "interval_count": 1,
    "livemode": false,
    "metadata": {},
    "nickname": "Netflix - Individual Plan: $6.99 billed monthly",
    "object": "plan",
    "product": "prod_Ek5x7DjLjWH0Uu",
    "tiers": null,
    "tiers_mode": null,
    "transform_usage": null,
    "trial_period_days": null,
    "usage_type": "licensed"
  },
  "quantity": 1,
  "subscription": "sub_Ek0NBc7clsXX73"
}